# Imersão dados

### Cruzando bases de dados

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dados = pd.read_csv("https://gist.githubusercontent.com/tgcsantos/3bdb29eba6ce391e90df2b72205ba891/raw/22fa920e80c9fa209a9fccc8b52d74cc95d1599b/dados_imoveis.csv")
dados[["moeda", "Valor_anuncio", "Tipo_anuncio"]] = dados["Valor"].str.split(expand=True)

dados_vendas = dados[dados["Tipo_anuncio"].isnull()]
dados_vendas["Valor_anuncio_float"] = dados_vendas["Valor_anuncio"].str.replace(".", "").astype(float)
dados_vendas["Valor_m2"] = dados_vendas["Valor_anuncio_float"] / dados_vendas["Metragem"]

dados_bairro = dados_vendas.groupby("Bairro").sum()
dados_bairro["Valor_m2_bairro"] = dados_bairro["Valor_anuncio_float"]/dados_bairro["Metragem"]

top_bairros = dados_vendas["Bairro"].value_counts()[:10].index

lendo a base de dados com o encoding ISO-8859-1 (pontuação), sep (separador), thousands(milhar), decimal(, do brasil)
basicamente dropa todas as colunas que forem vazias com o axis=1

In [ ]:
ibge_sp = pd.read_csv("https://gist.githubusercontent.com/tgcsantos/85f8c7b0a2edbc3e27fcad619b37d886/raw/a4954781e6bca9cb804062a3eea0b3b84679daf4/Basico_SP1.csv", encoding='ISO-8859-1', sep=';', thousands='.', decimal=',')
ibge_sp.dropna(how='all', axis=1, inplace=True)
ibge_sp.head()


In [ ]:
ibge_sp.info()

In [ ]:
# enderecos = pd.read_csv(".\dados\enderecos.csv")
enderecos.head()

In [ ]:
enderecos_sp = enderecos.query("sigla_uf == 'SP'")
enderecos_sp.head()

In [ ]:
enderecos_sp["rua"] = enderecos_sp["tipo_logr"] + " " + enderecos_sp["logr_nome"]
enderecos_sp["rua"] = enderecos_sp["rua"].str.lower().str.strip() #strip tira espaços
enderecos_sp.head()

Regex -> extrai tudo que for letra mais um espaço (remover numero e deixar apenas o nome das ruas)

In [ ]:
dados_vendas["apenas_ruas"] = dados_vendas["Rua"].str.extract(r'(^[\w ]+)')
dados_vendas["apenas_ruas"] = dados_vendas["apenas_ruas"].str.lower().str.strip()
dados_vendas.sample(5)

Left Join dados_vendas com o enderecos_sp e removendo informações duplicadas relacionadas ao dados_vendas

In [ ]:
dados_geo = pd.merge(left=dados_vendas, right=enderecos_sp[["rua", "cep", "latitude", "longitude"]], how="left", left_on="apenas_ruas", right_on="rua").drop_duplicates(subset=dados_vendas.columns).query("cep > 0")
dados_geo.head()

```bash
pip install shapely
```

In [ ]:
from shapely.geometry import Point
latitude = -23.56
longitude = -46.59
Point(longitude, latitude)

In [ ]:
from shapely.geometry import Polygon
Polygon([[0, 0], [1, 0], [1, 1], [0, 1], [0, 0]])

In [ ]:
Polygon([[0, 0], [1, 0], [1, 1], [0, 1], [0, 0]]).contains(Point(0.1,0.9))

como instalar o geopandas [resposta do Hicham Zouarhi](https://gis.stackexchange.com/questions/330840/error-installing-geopandas)

```bash
pip install ./install-geopandas/GDAL-3.4.3-cp310-cp310-win_amd64.whl
pip install ./install-geopandas/Fiona-1.8.21-cp310-cp310-win_amd64.whl
pip install geopandas
```
instale <strong>gdal</strong> levando em conta a versão do seu python e a arquitetura do seu windows (32 ou 64bits) <br/>
instale <strong>Fiona</strong> levando em conta a versão do seu python e a arquitetura do seu windows (32 ou 64bits)<br/>
instale <strong>geopandas</strong>

In [ ]:
import geopandas as gpd
# setor_censo = gpd.read_file('./dados/35SEE250GC_SIR.shp')
# setor_censo.head()

In [ ]:
setor_censo_sp = setor_censo[setor_censo.NM_MUNICIP == "SÃO PAULO"]
setor_censo_sp[setor_censo_sp.contains(Point(-46.63, -23.58))] #Base alura

In [ ]:
dados_geo["Point"] = ""
for i in dados_geo.index:
    dados_geo["Point"][i] = Point(dados_geo["longitude"][i], dados_geo["latitude"][i])

dados_geo['setor_censo'] = dados_geo["Point"][:10].map(
    lambda x: setor_censo_sp.loc[setor_censo_sp.contains(x), 'CD_GEOCODI'].values
).str[0]
# dados_geo

In [ ]:
# dados_geo = pd.read_csv("./dados/dados_geo.csv")
dados_geo.head()

In [ ]:
dados_vendas_censo = pd.merge(
    left=dados_geo,
    right=ibge_sp,
    how="left",
    left_on="setor_censo",
    right_on="Cod_setor"
)
dados_vendas_censo.head()

In [ ]:
dados_vendas_censo.info()

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(data = dados_vendas_censo, x="V005", y="Valor_m2")